# Extrayendo los datos con News API
 

In [ ]:
import pandas as pd
Df_Original=pd.read_csv('../Datos/noticias_bitcoin_top3_diarias.csv')
Df_Original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fecha        141 non-null    object
 1   titulo       141 non-null    object
 2   descripcion  141 non-null    object
 3   contenido    141 non-null    object
 4   url          141 non-null    object
 5   fuente       141 non-null    object
dtypes: object(6)
memory usage: 6.7+ KB


## Primer paquete de fechas

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

API_KEY = '1ff696647723d52c49e073a642c3fd30'  # Reemplaza esto con tu clave de GNews

def obtener_noticias(fecha):
    """
    Obtiene hasta 3 noticias para una fecha específica
    """
    url = 'https://gnews.io/api/v4/search'
    params = {
        'q': 'bitcoin',
        'lang': 'en',
        'from': f'{fecha}T00:00:00Z',
        'to': f'{fecha}T23:59:59Z',
        'max': 2,  # máximo 3 noticias por día
        'token': API_KEY
    }

    response = requests.get(url, params=params)
    noticias = []

    if response.status_code == 200:
        data = response.json()
        for article in data.get('articles', []):
            noticias.append({
                'fecha': fecha,
                'titulo': article['title'],
                'descripcion': article['description'],
                'contenido': article.get('content', ''),
                'url': article['url'],
                'fuente': article['source']['name']
            })
    else:
        print(f"Error en {fecha}: {response.status_code}")
    
    return noticias

# Obtener noticias de los últimos 100 días
noticias_totales = []
oy = hoy = datetime(2025, 4, 5) # 5 de abril de 2025
for i in range(100):
    fecha = (hoy - timedelta(days=i)).strftime('%Y-%m-%d')
    print(f'📅 Obteniendo noticias para {fecha}...')
    noticias = obtener_noticias(fecha)
    noticias_totales.extend(noticias)
    time.sleep(1)  # Evita sobrecargar la API gratuita

# Guardar en un CSV
df = pd.DataFrame(noticias_totales)
print("✅ ¡Noticias guardadas en noticias_bitcoin_gnews.csv!")

https://www.bbc.com/mundo/topics/cg726106pdgt
https://www.bbc.com/urdu/articles/c8dq26rve7vo
https://www.bbc.com/pidgin/articles/c3gj2d2n5elo




## Segundo paquete de fechas

In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

API_KEY = '1ff696647723d52c49e073a642c3fd30'  # Reemplaza esto con tu clave de GNews

def obtener_noticias(fecha):
    """
    Obtiene hasta 3 noticias para una fecha específica
    """
    url = 'https://gnews.io/api/v4/search'
    params = {
        'q': 'bitcoin',
        'lang': 'en',
        'from': f'{fecha}T00:00:00Z',
        'to': f'{fecha}T23:59:59Z',
        'max': 2,  # máximo 3 noticias por día
        'token': API_KEY
    }

    response = requests.get(url, params=params)
    noticias = []

    if response.status_code == 200:
        data = response.json()
        for article in data.get('articles', []):
            noticias.append({
                'fecha': fecha,
                'titulo': article['title'],
                'descripcion': article['description'],
                'contenido': article.get('content', ''),
                'url': article['url'],
                'fuente': article['source']['name']
            })
    else:
        print(f"Error en {fecha}: {response.status_code}")
    
    return noticias

# Obtener noticias de los últimos 100 días
noticias_totales = []
hoy = hoy = datetime(2025, 1, 4) # 4 de enero de 2025
for i in range(100):
    fecha = (hoy - timedelta(days=i)).strftime('%Y-%m-%d')
    print(f'📅 Obteniendo noticias para {fecha}...')
    noticias = obtener_noticias(fecha)
    noticias_totales.extend(noticias)
    time.sleep(1)  # Evita sobrecargar la API gratuita

# Guardar en un CSV
df_2 = pd.DataFrame(noticias_totales)
print("✅ ¡Noticias guardadas en noticias_bitcoin_gnews.csv!")

📅 Obteniendo noticias para 2025-01-04...
📅 Obteniendo noticias para 2025-01-03...
📅 Obteniendo noticias para 2025-01-02...
📅 Obteniendo noticias para 2025-01-01...
📅 Obteniendo noticias para 2024-12-31...
📅 Obteniendo noticias para 2024-12-30...
📅 Obteniendo noticias para 2024-12-29...
📅 Obteniendo noticias para 2024-12-28...
📅 Obteniendo noticias para 2024-12-27...
📅 Obteniendo noticias para 2024-12-26...
📅 Obteniendo noticias para 2024-12-25...
📅 Obteniendo noticias para 2024-12-24...
📅 Obteniendo noticias para 2024-12-23...
📅 Obteniendo noticias para 2024-12-22...
📅 Obteniendo noticias para 2024-12-21...
📅 Obteniendo noticias para 2024-12-20...
📅 Obteniendo noticias para 2024-12-19...
📅 Obteniendo noticias para 2024-12-18...
📅 Obteniendo noticias para 2024-12-17...
📅 Obteniendo noticias para 2024-12-16...
📅 Obteniendo noticias para 2024-12-15...
📅 Obteniendo noticias para 2024-12-14...
📅 Obteniendo noticias para 2024-12-13...
📅 Obteniendo noticias para 2024-12-12...
📅 Obteniendo not

In [7]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fecha        182 non-null    object
 1   titulo       182 non-null    object
 2   descripcion  182 non-null    object
 3   contenido    182 non-null    object
 4   url          182 non-null    object
 5   fuente       182 non-null    object
dtypes: object(6)
memory usage: 8.7+ KB


In [8]:
df=pd.concat([Df_Original,df_2],ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fecha        323 non-null    object
 1   titulo       323 non-null    object
 2   descripcion  323 non-null    object
 3   contenido    323 non-null    object
 4   url          323 non-null    object
 5   fuente       323 non-null    object
dtypes: object(6)
memory usage: 15.3+ KB


In [ ]:
df.to_csv('../Datos/noticias_bitcoin_top3_diarias.csv', index=False)#Reemplaza con la ruta de los datos
print(df.head())

        fecha                                             titulo  \
0  2025-04-07        Bitcoin price falls amid tariff market rout   
1  2025-04-07  US crypto stocks plunge as bitcoin hits new 20...   
2  2025-04-05  SpaceX tourists return to Earth after historic...   
3  2025-04-04   BlackRock buys $66M in Bitcoin amid market crash   
4  2025-04-04  Markets tumble as Trump launches ‘Unstoppable’...   

                                         descripcion  \
0  Bitcoin, the world's largest cryptocurrency by...   
1  U.S.-listed shares of crypto companies tumbled...   
2  Bitcoin investor Chun Wang took three people o...   
3  BlackRock acquired $66 million in Bitcoin on t...   
4  Markets fell after Trump announced new tariffs...   

                                           contenido  \
0  The price of bitcoin (BTC-USD), the world's la...   
1  (Reuters) - U.S.-listed shares of crypto compa...   
2  Four space tourists have safely arrived back o...   
3  The sell-off in the stock m

##  Extraccion de los datos historicos de bitcoin

In [1]:
pip install yfinance


     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB 640.0 kB/s eta 0:00:05
      --------------------------------------- 0.0/3.0 MB 487.6 kB/s eta 0:00:07
     -- ------------------------------------- 0.2/3.0 MB 1.1 MB/s eta 0:00:03
     ------- -------------------------------- 0.6/3.0 MB 3.2 MB/s eta 0:00:01
     -------------- ------------------------- 1.1/3.0 MB 5.3 MB/s eta 0:00:01
     --------------------- ------------------ 1.6/3.0 MB 6.0 MB/s eta 0:00:01
     ---------------------------- ----------- 2.2/3.0 MB 7.3 MB/s eta 0:00:01
     ----------------------------------- ---- 2.7/3.0 MB 7.8 MB/s eta 0:00:01
     ---------------------------------------  3.0/3.0 MB 8.0 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 7.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies:

In [4]:
import yfinance as yf
import pandas as pd

# Descargar datos históricos de BTC/USD
btc_data = yf.download("BTC-USD", start="2022-04-06", end="2025-04-06", interval="1d")

# Verificar los primeros registros
print("Datos descargados:")
print(btc_data.head())

# Verificar si hay datos nulos o problemas
print("\nResumen de los datos:")
print(btc_data.info())

# Restablecer el índice para que la fecha sea una columna
btc_data.reset_index(inplace=True)

# Verificar la estructura después de restablecer el índice
print("\nDatos después de restablecer el índice:")
print(btc_data.head())

# Guardar el archivo en CSV

btc_data.to_csv("C:/Repos/Projecto_TRM/Datos/btc_USD_historic.csv", index=False)

print("\nArchivo CSV generado correctamente.")


[*********************100%***********************]  1 of 1 completed

Datos descargados:
Price              Close          High           Low          Open  \
Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Date                                                                 
2022-04-06  43206.738281  45544.355469  43193.953125  45544.355469   
2022-04-07  43503.847656  43860.699219  42899.906250  43207.500000   
2022-04-08  42287.664062  43903.019531  42183.289062  43505.136719   
2022-04-09  42782.136719  42786.816406  42183.253906  42282.078125   
2022-04-10  42207.671875  43376.378906  42021.207031  42781.089844   

Price            Volume  
Ticker          BTC-USD  
Date                     
2022-04-06  39393395788  
2022-04-07  26101973106  
2022-04-08  27215995394  
2022-04-09  16050772496  
2022-04-10  17654475582  

Resumen de los datos:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2022-04-06 to 2025-04-05
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------      